In [5]:
import asyncio
from typing import Annotated, TypedDict

from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.store.memory import InMemoryStore
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI

In [2]:
# ---------- MEMORY STORE ----------
# This is shared across users
store = InMemoryStore()

# Add user-specific context into store
store.put(
    namespace=("user123", "chitchat"),
    key="profile",
    value={
        "rules": [
            "User likes short, direct language",
            "User only speaks English & Python"
        ],
        "name": "Alice",
    },
)

store.put(
    namespace=("user456", "chitchat"),
    key="profile",
    value={
        "rules": ["User prefers concise answers"],
        "name": "Bob",
    },
)

In [6]:

class State(TypedDict):
    messages: Annotated[list, "The conversation so far"]

llm = ChatOpenAI(model="gpt-4o-mini")

In [7]:
async def agent_node(state: State, config):
    """Instead of hardcoding logic, just inject memory into the LLM prompt"""
    user_id = config["configurable"].get("user_id")
    namespace = (user_id, "chitchat")

    user_profile = store.get(namespace, "profile") or {}
    memory_context = f"User profile: {user_profile}"

    # Let the LLM decide the answer
    response = await llm.ainvoke(
        state["messages"] + [HumanMessage(content=memory_context)]
    )

    return {"messages": state["messages"] + [response]}

In [8]:
workflow = StateGraph(State)
workflow.add_node("agent", agent_node)
workflow.add_edge(START, "agent")
workflow.add_edge("agent", END)

memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)

In [10]:
async def main():
    print("\n--- User123 ---")
    result = await graph.ainvoke(
        {"messages": [HumanMessage(content="What is my name?")]},
        config={"configurable": {"user_id": "user123", "thread_id": 1}}
    )
    print(result["messages"][-1].content)

    result = await graph.ainvoke(
        {"messages": [HumanMessage(content="What do I like?")]},
        config={"configurable": {"user_id": "user123", "thread_id": 1}}
    )
    print(result["messages"][-1].content)

    print("\n--- User456 ---")
    result = await graph.ainvoke(
        {"messages": [HumanMessage(content="What is my name?")]},
        config={"configurable": {"user_id": "user456", "thread_id": 1}}
    )
    print(result["messages"][-1].content)

    result = await graph.ainvoke(
        {"messages": [HumanMessage(content="What do I like?")]},
        config={"configurable": {"user_id": "user456", "thread_id": 1}}
    )
    print(result["messages"][-1].content)

await(main())


--- User123 ---
Your name is Alice.
Based on your profile, you like short and direct language. You also communicate in English and Python. If there's anything specific you'd like to discuss or know more about, just let me know!

--- User456 ---
Your name is Bob.
Hi Bob! Since you prefer concise answers, I'll keep it brief. If you'd like to tell me about your interests or hobbies, I can help you figure out what you like! Otherwise, I can provide some common interests if that helps.
